In [8]:
# connector de python à mysql
import mysql.connector
import json
import re
import dateparser
import datetime

#**************************************************************************************CREATION DE LA CONNEXION SQLALCHEMY**********************************************************

## IMPORT DU CONFIG.JSON
# assignation de la config.json à fichierConfig
fichierConfig = "config.json"
# ouverture et chargement des donnée contenu dans fichierConfig
with open(fichierConfig) as fichier:
    config = json.load(fichier)["mysql"]

# assignation de de la connexion par create_engine avec les éléement de connexion + les info du fichierConfig à engine
# en dehors de la class car cet élémeent ne change pas
connection = mysql.connector.connect(host=config["host"],database = config["bdd"],user=config["user"],password=config["password"])
cursor = connection.cursor()

In [10]:
msg = ''
result = ''
salle_valide = ''
session = 40

mail_contenu = "Salut, J’ai besoin de la salle 736 le 17 janvier entre 8 h et 10 h"

            
            
            #********************************** Traitement du mail en reservation***********************
            
mail_contenu = "Salut, J’ai besoin de la salle 736 le 17 janvier entre 8 h et 10 h"
mail_contenu = str(mail_contenu)       ## on converti "mail_contenu" en chaine de caractere avec str()

            ####################################################### Motif pour la Salle ##########################################
motif_salle = "(?:salle+\s+\d+)|(?:Salle+\s+\d+)"

            ####################################################### Motif pour la Date ##########################################

motif_date = "(?:\d+\s+janvier)|(?:\d+\s+fevrier)|(?:\d+\s+mars)|(?:\d+\s+avril)|(?:\d+\s+mai)|(?:\d+\s+juin)|(?:\d+\s+juillet)|(?:\d+\s+août)|(?:\d+\s+septembre)|(?:\d+\s+octobre)|(?:\d+\s+novembre)|(?:\d+\s+decembre)|(?:\d+\s+Janvier)|(?:\d+\s+Fevrier)|(?:\d+\s+Mars)|(?:\d+\s+Avril)|(?:\d+\s+Mai)|(?:\d+\s+Juin)|(?:\d+\s+Juillet)|(?:\d+\s+Août)|(?:\d+\s+Septembre)|(?:\d+\s+Octobre)|(?:\d+\s+Novembre)|(?:\d+\s+Decembre)|(?:\d+\s+January)|(?:\d+\s+February)|(?:\d+\s+March)|(?:\d+\s+April)|(?:\d+\s+May)|(?:\d+\s+June)|(?:\d+\s+July)|(?:\d+\s+Augut)|(?:\d+\s+September)|(?:\d+\s+October)|(?:\d+\s+November)|(?:\d+\s+December)|(?:\d+\s+january)|(?:\d+\s+february)|(?:\d+\s+march)|(?:\d+\s+april)|(?:\d+\s+Mai)|(?:\d+\s+june)|(?:\d+\s+july)|(?:\d+\s+august)|(?:\d+\s+september)|(?:\d+\s+october)|(?:\d+\s+november)|(?:\d+\s+december)"  #### (?:\d+.\d+) 12/03 pour la date format numérique

            ####################################################### Motif pour les horraire ##########################################

motif_horraire = "(?:\d+.Heures)|(?:\d+.Heure)|(?:\d+.heures)|(?:\d+.heure)|(?:\d+.h)|(?:\d+.H)" #informations dans "mail_contenu"

            ####################################################### Salle reserver ##########################################


res_mail_salle = re.findall(motif_salle,mail_contenu) # application de la méthode findall() avec regex, on applique le motif sur le "mail_contenu", stocker dans la variable "res_mail"
num_salle_reserv = str(res_mail_salle[0])
num_salle_reserv = str(num_salle_reserv)

            ####################################################### Date reserver ##########################################

res_mail_date = re.findall(motif_date,mail_contenu) # application de la méthode findall() avec regex, on applique le motif sur le "mail_contenu", stocker dans la variable "res_mail"

date_reserv = str(res_mail_date[0])
date_reserv = str(date_reserv)

            ####################################################### HEURE reserver ##########################################


res_mail_horraire = re.findall(motif_horraire,mail_contenu) # application de la méthode findall() avec regex, on applique le motif sur le "mail_contenu", stocker dans la variable "res_mail"
res_mail_horraire
debut_reserv = res_mail_horraire[0][0:2]
debut_reserv = int(debut_reserv)

fin_reserv = res_mail_horraire[1][0:2]
fin_reserv = int(fin_reserv)   
time_reserv = fin_reserv - debut_reserv 

  
date_reserv = dateparser.parse(date_reserv)  
now = datetime.datetime.now()   
if date_reserv <= now:
    now_year = now.year + 1     
else:                           
    now_year = now.year                   
    date_reserv = str(date_reserv)  
    date_reserv = date_reserv[5:10] 
    date_reserv = str(now_year) + ' ' + date_reserv  
    date_reserv = dateparser.parse(date_reserv)
    date_reserv = date_reserv.date()   
           
if debut_reserv in range(0,25) and fin_reserv in range(0,25):
    debut_reserv = debut_reserv 
    fin_reserv = fin_reserv     
else:
    debut_reserv = False        
    fin_reserv = False          ## l'heure fin n'est pas bonne
                
                

            #********************************** Check de la dispo de la demande de reservation***********************
            
cursor.execute('SELECT sa_name FROM SALLE WHERE sa_name LIKE %s;', (num_salle_reserv,))
resa_db_salle = cursor.fetchone()
if resa_db_salle == None:
    salle_valide = False
    msg = "Salle non valide"
    print('None',msg)
else:
    msg = "Salle valide"
    salle_valide = True
    print("Il y as une salle")
            
            ## ON CHECK SI UNE RESERVATION EXISTE DEJA DANS LA BASE DE DONNEE POUR LE MEME "nom de salle" la même "DATE" et si les heures peuvent correspondre
            ## ON STOCKE LE RESULTAT DE LA REQUETE DANS la variable "resa_db"
            
            ## SI LE RESULTAT DE LA REQUETE N'AS AUCUNNE CORRESPONDANCE et que les heure debut/fin son valide ALORS 
            

cursor.execute('SELECT res_id, res_date, res_heure_arrive, res_heure_depart, S.sa_name FROM RESERVATION R INNER JOIN SALLE S ON S.sa_id=R.sa_id WHERE S.sa_name=%s AND R.res_date=%s AND(( %s BETWEEN R.res_heure_arrive AND R.res_heure_depart ) OR (%s BETWEEN R.res_heure_arrive AND R.res_heure_depart));', (num_salle_reserv,date_reserv,debut_reserv,fin_reserv))

resa_db_1 = cursor.fetchone()

            #print(resa_db_1)

if resa_db_1 == None:
    result = True#,"TESTING OK FIRST"
    print(result)

###################################################### TEST DU SI L'HEURE  DE DEBUT DEMANDER CORRESPOND A UNE HEURE DE FIN DEJA EXISTANTE#####################
elif resa_db_1 != None and debut_reserv == resa_db_1[3]:
    cursor.execute('SELECT * FROM RESERVATION R INNER JOIN SALLE S ON S.sa_id=R.sa_id WHERE S.sa_name=%s AND R.res_date=%s AND ((%s > R.res_heure_arrive AND %s < R.res_heure_depart) OR (%s >= R.res_heure_arrive AND %s <= R.res_heure_depart));', (num_salle_reserv,date_reserv,debut_reserv,debut_reserv, fin_reserv,fin_reserv))
    result_fin_to_debut_reserv = cursor.fetchone()
    
    if result_fin_to_debut_reserv == None and resa_db_1 != None:
        result = True#,"TESTING IS OK 1"
        print(result)
    


########################################################### TEST DU SI L'HEURE DE FIN DEMANDER CORRESPOND A UNE HEURE DE DEBUT DEJA EXISTANTE#################

elif resa_db_1 != None and fin_reserv == resa_db_1[2]:
    cursor.execute('SELECT * FROM RESERVATION R INNER JOIN SALLE S ON S.sa_id=R.sa_id WHERE S.sa_name=%s AND R.res_date=%s AND ((%s >= R.res_heure_arrive AND %s <= R.res_heure_depart) OR (%s > R.res_heure_arrive AND %s < R.res_heure_depart));', (num_salle_reserv,date_reserv,debut_reserv,debut_reserv, fin_reserv,fin_reserv))
    result_debut_to_fin_reserv = cursor.fetchone()
    
    if result_debut_to_fin_reserv == None and resa_db_1 != None:
        result = True#,"TESTING IS OK 2"
        print(result)


else:
    result = False#,"TESTING NOT OK 1"
    print(result)




if result == True and salle_valide == True:
    
    parameter_ma_contenu = mail_contenu ## on initialise le mail_contenu dans la variable "parameter_ma_contenu"
    parameter_session_id = session ## on initialise le session['id'] qui est l'id de la session en cours dans la variable "parameter_session_id"

    cursor.callproc("PI_MAIL_SIMPLE", [parameter_ma_contenu,parameter_session_id,],)    ## ON INSERE DANS LA TABLE MAIL LE MAIL CONTENU ET L'ID DE LA SESSION
    connection.commit() ## ON VALIDE L'INSERTION
               
                ### recup ma_id du mail inserer dans le dernier ma_contenu de l'user
    cursor.execute('SELECT ma_id FROM MAIL WHERE in_id = %s ORDER BY ma_date DESC LIMIT 1 ', (parameter_session_id,))   ## ON RECUPERE LE DERNIER MAIL ENVOYER DE L'USER ORDONNER PAR LA DATE ET L'HEURE ET SECONDE LA PLUS RECENTE 
    ma_id_recup = cursor.fetchone() ## ON STOCKE LE RESULTAT DE LA REQUETE DANS LA VARIABLE "ma_id_recup"
    ma_id_recup = ma_id_recup[0]    ## ma_id_recup est initialise avec son premier élément de sa liste
               
                
                ### recup le sa_id where num_salle_reserv
    cursor.execute('SELECT sa_id FROM SALLE WHERE sa_name = %s', (num_salle_reserv,)) ## ON REQUETE LA DATABASE POUR RECUPER L'ID DE LA SALLE DEMANDER PAR LE NOM DE LA SALLE
    sa_id_recup = cursor.fetchone() # on stocke le resultat de la requete dans "sa_id_recup"
    sa_id_recup = str(sa_id_recup[0])    # sa_id_recup est initialise avec son premier élément de sa liste



             #********************************** Insertion du contenu en reservation dans RESERVATION***********************
    args1 = date_reserv
    args2 = debut_reserv
    args3 = fin_reserv
    args4 = time_reserv
    args5 = sa_id_recup
    args6 = ma_id_recup
    
    #args = [date_reserv,debut_reserv,fin_reserv,time_reserv,sa_id_recup,ma_id_recup,]      
                #args = [date_reserv,debut_reserv,fin_reserv,time_reserv,sa_id_recup,ma_id_recup,] ### ASSIGNATION DANS "args" LISTE DES ARGUMENT D'ENTRER DANS LA PROCEDURE SOTCKE POUR INSERER LA RESERVATION
    cursor.callproc("PI_RES_SIMPLE", (args1,args2,args3,args4,args5,args6,))    ### EXECUTION DE LA PROCEDURE STOCKE POUR INSERER LA RESERVATION
    connection.commit()     ## ON VALIDE L'INSERTION
                
                # fetch result parameters
                
                

    msg = 'You have successfully send your reservation !'       ## ON AVERTIE L'USER qu'il a bien envoyer sa reservation 
    print(msg)

else:
    msg = 'your reservation is invalid!' ### AJOUTER CONDITION IF AND ELSE TEST POUR LE MSG SOIT DIFFERENT
    print(msg)

Il y as une salle
True


DatabaseError: 1366 (HY000): Incorrect integer value: '(139,)' for column 'SALLE_id' at row 1